# Animations

This notebook demonstrates how to create animations in MolViewSpec.
Animations allow you to interpolate properties over time, creating dynamic visualizations.

**Note**: Animation API is currently being implemented in the TypeScript version.
This notebook shows the expected API based on the Python implementation.

In [ ]:
import { createBuilder } from "../../molviewspec-ts/molviewspec/builder.ts";
import type { Snapshot } from "../../molviewspec-ts/molviewspec/nodes.ts";

## Color Animation

Create an animation that transitions a ligand color from blue to red.

**Expected API** (to be implemented):

In [ ]:
const builder = createBuilder();

const structure = builder
  .download('https://files.wwpdb.org/download/1cbs.cif')
  .parse('mmcif')
  .modelStructure();

// Add polymer representation
structure.component().representation();

// Add ligand with color reference
structure
  .component("ligand")
  .representation("ball_and_stick")
  .color("blue", undefined, undefined, "color");

// Note: Animation API to be implemented
// Expected usage:
// const anim = builder.animation();
// 
// anim.interpolate({
//   kind: "color",
//   target_ref: "color",
//   duration_ms: 1000,
//   property: "color",
//   palette: {
//     kind: "continuous",
//     colors: ["blue", "red"],
//   },
// });

// For now, we can create the snapshot without animation
const snapshot: Snapshot = builder.getSnapshot({
  title: "Color Animation Example",
  description: "Animating ligand color from blue to red",
  linger_duration_ms: 1000,
});

// Manually add animation node to demonstrate structure
// This shows what the animation structure would look like:
snapshot.animation = {
  kind: "animation",
  params: {
    frame_time_ms: 16.666666666666668,  // ~60 fps
    autoplay: true,
    loop: false,
    include_camera: false,
    include_canvas: false,
  },
  children: [
    {
      kind: "interpolate",
      params: {
        frequency: 1,
        alternate_direction: false,
        easing: "linear",
        target_ref: "color",
        property: "color",
        start_ms: 0,
        duration_ms: 1000,
        kind: "color",
        palette: {
          kind: "continuous",
          colors: ["blue", "red"],
        },
      },
    },
  ],
};

// Create States object
const statesData = {
  kind: "multiple" as const,
  metadata: {
    title: "Animation Test",
    description: "Color animation example",
    timestamp: new Date().toISOString(),
    version: "1.8",
  },
  snapshots: [snapshot],
};

console.log(JSON.stringify(statesData, null, 2));

## Animation Concepts

### Animation Node

The animation node controls overall animation parameters:
- `frame_time_ms`: Time per frame (16.67ms = 60 fps)
- `autoplay`: Whether to start animation automatically
- `loop`: Whether to loop the animation
- `include_camera`: Whether to animate camera
- `include_canvas`: Whether to animate canvas properties

### Interpolation Kinds

Different types of interpolation are available:

1. **Color**: Interpolate between colors
   ```typescript
   {
     kind: "color",
     palette: { kind: "continuous", colors: ["blue", "red"] }
   }
   ```

2. **Scalar**: Interpolate numerical values
   ```typescript
   {
     kind: "scalar",
     from: 0.0,
     to: 1.0
   }
   ```

3. **Vec3**: Interpolate 3D vectors (positions, directions)
   ```typescript
   {
     kind: "vec3",
     from: [0, 0, 0],
     to: [10, 10, 10]
   }
   ```

4. **Rotation Matrix**: Interpolate rotation matrices
   ```typescript
   {
     kind: "rotation_matrix",
     from: [1, 0, 0, 0, 1, 0, 0, 0, 1],
     to: [0, -1, 0, 1, 0, 0, 0, 0, 1]
   }
   ```

### Easing Functions

Control the timing curve of animations:
- `linear`: Constant speed
- `ease_in`: Start slow, accelerate
- `ease_out`: Start fast, decelerate
- `ease_in_out`: Slow at both ends
- `cubic`, `quadratic`, `exponential`: Various curves

## Opacity Animation Example

Animate the opacity of a component (expected API).

In [ ]:
const builder2 = createBuilder();

const struct2 = builder2
  .download('https://files.wwpdb.org/download/1cbs.cif')
  .parse('mmcif')
  .modelStructure();

struct2
  .component("polymer")
  .representation()
  .opacity(1.0, undefined, "opacity_node");

// Expected animation API:
// const anim2 = builder2.animation();
// 
// anim2.interpolate({
//   kind: "scalar",
//   target_ref: "opacity_node",
//   property: "opacity",
//   duration_ms: 2000,
//   from: 1.0,
//   to: 0.2,
//   easing: "ease_in_out",
// });

const snapshot2 = builder2.getSnapshot({
  title: "Opacity Animation",
  description: "Fade polymer in and out",
  linger_duration_ms: 2000,
});

// Manually add animation for demonstration
snapshot2.animation = {
  kind: "animation",
  params: {
    frame_time_ms: 16.666666666666668,
    autoplay: true,
    loop: true,  // Loop this animation
  },
  children: [
    {
      kind: "interpolate",
      params: {
        target_ref: "opacity_node",
        property: "opacity",
        start_ms: 0,
        duration_ms: 2000,
        kind: "scalar",
        from: 1.0,
        to: 0.2,
        easing: "ease_in_out",
        alternate_direction: true,  // Fade in and out
      },
    },
  ],
};

console.log(JSON.stringify(snapshot2, null, 2));

## Camera Animation Example

Animate camera position to create a rotating view.

In [ ]:
const builder3 = createBuilder();

const struct3 = builder3
  .download('https://files.wwpdb.org/download/1cbs.cif')
  .parse('mmcif')
  .modelStructure();

struct3.component().representation();

// Set initial camera position
builder3.camera(
  {
    target: [0, 0, 0],
    position: [50, 0, 0],
    up: [0, 1, 0],
  },
  undefined,
  "camera"
);

// Expected animation API:
// const anim3 = builder3.animation({ include_camera: true });
// 
// anim3.interpolate({
//   kind: "vec3",
//   target_ref: "camera",
//   property: "position",
//   duration_ms: 5000,
//   from: [50, 0, 0],
//   to: [0, 0, 50],
//   easing: "linear",
// });

const snapshot3 = builder3.getSnapshot({
  title: "Camera Animation",
  description: "Rotating camera view",
  linger_duration_ms: 5000,
});

// Manually add animation for demonstration
snapshot3.animation = {
  kind: "animation",
  params: {
    frame_time_ms: 16.666666666666668,
    autoplay: true,
    loop: true,
    include_camera: true,  // Enable camera animation
  },
  children: [
    {
      kind: "interpolate",
      params: {
        target_ref: "camera",
        property: "position",
        start_ms: 0,
        duration_ms: 5000,
        kind: "vec3",
        from: [50, 0, 0],
        to: [0, 0, 50],
        easing: "linear",
      },
    },
  ],
};

console.log(JSON.stringify(snapshot3, null, 2));

## Multiple Simultaneous Animations

You can have multiple interpolations running at the same time.

In [ ]:
const builder4 = createBuilder();

const struct4 = builder4
  .download('https://files.wwpdb.org/download/1cbs.cif')
  .parse('mmcif')
  .modelStructure();

// Polymer with color and opacity references
struct4
  .component("polymer")
  .representation()
  .color("blue", undefined, undefined, "poly_color")
  .opacity(1.0, undefined, "poly_opacity");

// Ligand with different animation
struct4
  .component("ligand")
  .representation("ball_and_stick")
  .color("red", undefined, undefined, "lig_color");

// Expected API:
// const anim4 = builder4.animation();
// 
// // Animate polymer color
// anim4.interpolate({
//   kind: "color",
//   target_ref: "poly_color",
//   property: "color",
//   duration_ms: 3000,
//   palette: { kind: "continuous", colors: ["blue", "cyan"] },
// });
// 
// // Animate polymer opacity
// anim4.interpolate({
//   kind: "scalar",
//   target_ref: "poly_opacity",
//   property: "opacity",
//   duration_ms: 3000,
//   from: 1.0,
//   to: 0.5,
//   easing: "ease_in_out",
// });
// 
// // Animate ligand color (delayed start)
// anim4.interpolate({
//   kind: "color",
//   target_ref: "lig_color",
//   property: "color",
//   start_ms: 1000,  // Start after 1 second
//   duration_ms: 2000,
//   palette: { kind: "continuous", colors: ["red", "yellow"] },
// });

const snapshot4 = builder4.getSnapshot({
  title: "Multiple Animations",
  description: "Animating color and opacity simultaneously",
  linger_duration_ms: 4000,
});

// Manually add multiple animations for demonstration
snapshot4.animation = {
  kind: "animation",
  params: {
    frame_time_ms: 16.666666666666668,
    autoplay: true,
    loop: false,
  },
  children: [
    {
      kind: "interpolate",
      params: {
        target_ref: "poly_color",
        property: "color",
        start_ms: 0,
        duration_ms: 3000,
        kind: "color",
        palette: { kind: "continuous", colors: ["blue", "cyan"] },
      },
    },
    {
      kind: "interpolate",
      params: {
        target_ref: "poly_opacity",
        property: "opacity",
        start_ms: 0,
        duration_ms: 3000,
        kind: "scalar",
        from: 1.0,
        to: 0.5,
        easing: "ease_in_out",
      },
    },
    {
      kind: "interpolate",
      params: {
        target_ref: "lig_color",
        property: "color",
        start_ms: 1000,
        duration_ms: 2000,
        kind: "color",
        palette: { kind: "continuous", colors: ["red", "yellow"] },
      },
    },
  ],
};

console.log(JSON.stringify(snapshot4, null, 2));

## Implementation Notes

### Current Status

The animation API shown in this notebook represents the expected interface based on
the Python implementation. The TypeScript builder will need to implement:

1. **Animation Builder Class**: Similar to other builder classes
   ```typescript
   export class Animation extends Base {
     interpolate(params: InterpolationParams): Animation { ... }
   }
   ```

2. **Animation Method on Root**: Add to Root class
   ```typescript
   animation(params?: AnimationParams): Animation { ... }
   ```

3. **Snapshot Integration**: Link animations to snapshots
   ```typescript
   getSnapshot(options): Snapshot {
     // Include animation node if present
   }
   ```

### Using Animations

Once implemented, animations will be used in the Mol* Stories viewer:

1. Create a snapshot with animated properties
2. Export as MVSJ format
3. Load in Mol* Stories viewer UI
4. Animations will play automatically or on user interaction

### Animation Properties

Animatable properties include:
- **Color**: Component colors
- **Opacity**: Component transparency
- **Camera**: Position, target, up vector
- **Canvas**: Background color
- **Transform**: Rotation and translation matrices
- **Clip**: Clipping plane/sphere/box parameters